The file "sqlite_db_pythonsqlite.db" is an export from PHPMyAdmin. It should be placed in the same folder as this notebook.

In [1]:
# Set up SQLite connection. uri=True allows read-only mode.
import sqlite3
import pandas as pd
file = "sqlite_db_pythonsqlite.db"
conn = sqlite3.connect('file:'+file+'?mode=ro', uri=True)

The first nine questions were answered in PHPMyAdmin. I have included those solutions below as well for ease of access.

**Q1.** Some of the facilities charge a fee to members, but some do not. Write a SQL query to produce a list of the names of the facilities that do.

**A1.** ```SELECT name FROM Facilities WHERE membercost != 0;```

In [2]:
pd.read_sql_query("SELECT name FROM Facilities WHERE membercost != 0;", conn)

,name
0,Tennis Court 1
1,Tennis Court 2
2,Massage Room 1
3,Massage Room 2
4,Squash Court


There are five such facilities: Tennis Courts 1 and 2, Massage Rooms 1 and 2, and the Squash Court.

**Q2.** How many facilities do not charge a fee to members?

**A2.** ```SELECT COUNT(*) FROM Facilities WHERE membercost = 0;```

In [3]:
pd.read_sql_query("SELECT COUNT(*) AS facility_count FROM Facilities WHERE membercost = 0;", conn)

,facility_count
0,4


There are four such facilities.

**Q3.** Write an SQL query to show a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost. Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

**A3.** 
```SELECT facid, 
    name, 
    membercost, 
    monthlymaintenance 
FROM Facilities 
WHERE membercost > 0 
    AND membercost < 0.2*monthlymaintenance;```

In [4]:
pd.read_sql_query("SELECT facid, name, membercost, monthlymaintenance FROM Facilities WHERE membercost > 0 AND membercost < 0.2*monthlymaintenance;", conn)

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,4,Massage Room 1,9.9,3000
3,5,Massage Room 2,9.9,3000
4,6,Squash Court,3.5,80


There are five such facilities, the same five as in the answer to the first question.

**Q4.** Write an SQL query to retrieve the details of facilities with ID 1 and 5. Try writing the query without using the OR operator.

**A4.** The previous question shows that these facilities both end with the number '2'. We use that to design the query.

```SELECT * FROM Facilities WHERE name LIKE '%2';```

In [5]:
pd.read_sql_query("SELECT * FROM Facilities WHERE name LIKE '%2';", conn)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


The observation was correct. The two facilities are Tennis Court 2 and Massage Room 2.

**Q5.** Produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100. Return the name and monthly maintenance of the facilities in question.

**A5.** 
```SELECT name, 
    monthlymaintenance, 
    CASE WHEN monthlymaintenance > 100 THEN 'expensive'
        ELSE 'cheap' END AS cost
FROM Facilities;```

In [6]:
pd.read_sql_query("SELECT name, monthlymaintenance, CASE WHEN monthlymaintenance > 100 THEN 'expensive' ELSE 'cheap' END AS cost FROM Facilities;", conn)

,name,monthlymaintenance,cost
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


Four facilities are indicated as "expensive": the two tennis courts and the two massage rooms. All other facilities are "cheap".

**Q6.** You'd like to get the first and last name of the last member(s) who signed up. Try not to use the LIMIT clause for your solution.

**A6.** ```SELECT firstname, surname FROM Members WHERE joindate IN (SELECT MAX(joindate) FROM Members);```

In [7]:
pd.read_sql_query("SELECT firstname, surname FROM Members WHERE joindate IN (SELECT MAX(joindate) FROM Members);", conn)

,firstname,surname
0,Darren,Smith


Darren Smith was the last member to join.

**Q7.** Produce a list of all members who have used a tennis court. Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name.

**A7.** For questions requesting the name formatted as a single column, we will format the name as "Surname, FirstName" in order to maintain the usual ordering. We add an additional ordering on the facility name to clean the output. We now that GUEST has member id 0 (see next question), so we use that to our advantage.
```SELECT DISTINCT
    CONCAT(Members.surname,', ',Members.firstname) AS member_name,
    Facilities.name AS facility_name
FROM (Bookings LEFT JOIN Members ON Bookings.memid = Members.memid) 
    LEFT JOIN Facilities ON Bookings.facid = Facilities.facid  
WHERE Facilities.name LIKE 'Tennis Court%' 
    AND Members.memid > 0
ORDER BY member_name, Facilities.name;```

SQLite doesn't recognize the `CONCAT` command, so we replace with `||`.

In [8]:
pd.read_sql_query("SELECT DISTINCT Members.surname || ', ' || Members.firstname AS member_name, Facilities.name AS facility_name FROM (Bookings LEFT JOIN Members ON Bookings.memid = Members.memid) LEFT JOIN Facilities ON Bookings.facid = Facilities.facid WHERE Facilities.name LIKE 'Tennis Court%' AND Members.memid > 0 ORDER BY member_name, Facilities.name;", conn)

,member_name,facility_name
0,"Bader, Florence",Tennis Court 1
1,"Bader, Florence",Tennis Court 2
2,"Baker, Anne",Tennis Court 1
3,"Baker, Anne",Tennis Court 2
4,"Baker, Timothy",Tennis Court 1
5,"Baker, Timothy",Tennis Court 2
6,"Boothe, Tim",Tennis Court 1
7,"Boothe, Tim",Tennis Court 2
8,"Butters, Gerald",Tennis Court 1
9,"Butters, Gerald",Tennis Court 2


Most members who reserved a tennis court reserved both at some point.

**Q8.** Produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30. Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.

**A8.** We include guest user's names as "Guest". 
```SELECT 
    Facilities.name,
    CASE WHEN Bookings.memid = 0 THEN 'Guest' 
        ELSE CONCAT(Members.surname, ', ', Members.firstname) END AS member_name,
    (Bookings.slots) 
        * (CASE WHEN Bookings.memid = 0 THEN Facilities.guestcost 
            ELSE Facilities.membercost END) AS cost
FROM (Bookings LEFT JOIN Facilities ON Bookings.facid = Facilities.facid) 
    LEFT JOIN Members ON Bookings.memid = Members.memid
WHERE DATE(Bookings.starttime) = '2012-09-14' 
HAVING cost > 30
ORDER BY cost DESC;```

PHPMyAdmin required the `HAVING` clause, which gives an error in SQLite. This error is fixed by placing the `cost > 30` condition within the `WHERE` clause.

In [9]:
pd.read_sql_query("SELECT Facilities.name, CASE WHEN Bookings.memid = 0 THEN 'Guest' ELSE Members.surname || ', ' || Members.firstname END AS member_name, (Bookings.slots) * (CASE WHEN Bookings.memid = 0 THEN Facilities.guestcost ELSE Facilities.membercost END) AS cost FROM (Bookings LEFT JOIN Facilities ON Bookings.facid = Facilities.facid) LEFT JOIN Members ON Bookings.memid = Members.memid WHERE DATE(Bookings.starttime) = '2012-09-14' AND cost > 30 ORDER BY cost DESC;", conn)

,name,member_name,cost
0,Massage Room 2,Guest,320.0
1,Massage Room 1,Guest,160.0
2,Massage Room 1,Guest,160.0
3,Massage Room 1,Guest,160.0
4,Tennis Court 2,Guest,150.0
5,Tennis Court 1,Guest,75.0
6,Tennis Court 1,Guest,75.0
7,Tennis Court 2,Guest,75.0
8,Squash Court,Guest,70.0
9,Massage Room 1,"Farrell, Jemima",39.6


Only one member spent over \\$30 in reservations.

**Q9.** This time, produce the same result as in Q8, but using a subquery.

**A9.** 
```SELECT
    FacilityBookings.name,
    CASE WHEN Members.memid = 0 THEN 'Guest' 
        ELSE CONCAT(Members.surname, ', ', Members.firstname) END AS member_name,
    slots * slot_cost AS cost
FROM
    (SELECT Facilities.name,
        Bookings.memid,
        Bookings.slots,
        CASE WHEN Bookings.memid = 0 THEN Facilities.guestcost 
            ELSE Facilities.membercost END AS slot_cost
    FROM Bookings LEFT JOIN Facilities ON Bookings.facid = Facilities.facid
    WHERE DATE(Bookings.starttime) = '2012-09-14') AS FacilityBookings 
    LEFT JOIN Members ON FacilityBookings.memid = Members.memid
HAVING cost > 30
ORDER BY cost DESC;```

We are required to make the same fixes as above. Since there is no `WHERE` clause in the original query, the `HAVING` clause is fully replaced by a new `WHERE` clause.

In [10]:
pd.read_sql_query("SELECT FacilityBookings.name, CASE WHEN Members.memid = 0 THEN 'Guest' ELSE Members.surname || ', ' || Members.firstname END AS member_name, slots * slot_cost AS cost FROM (SELECT Facilities.name, Bookings.memid, Bookings.slots, CASE WHEN Bookings.memid = 0 THEN Facilities.guestcost ELSE Facilities.membercost END AS slot_cost FROM Bookings LEFT JOIN Facilities ON Bookings.facid = Facilities.facid WHERE DATE(Bookings.starttime) = '2012-09-14') AS FacilityBookings LEFT JOIN Members ON FacilityBookings.memid = Members.memid WHERE cost > 30 ORDER BY cost DESC;", conn)

,name,member_name,cost
0,Massage Room 2,Guest,320.0
1,Massage Room 1,Guest,160.0
2,Massage Room 1,Guest,160.0
3,Massage Room 1,Guest,160.0
4,Tennis Court 2,Guest,150.0
5,Tennis Court 1,Guest,75.0
6,Tennis Court 1,Guest,75.0
7,Tennis Court 2,Guest,75.0
8,Squash Court,Guest,70.0
9,Massage Room 1,"Farrell, Jemima",39.6


The following questions were designed to be answered natively within the Jupyter Notebook by calling SQLite.

**Q10.** Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members! 

In [11]:
pd.read_sql_query("SELECT name, SUM(revenue) AS total_revenue FROM (SELECT Facilities.name, CASE WHEN memid = 0 THEN Facilities.guestcost ELSE Facilities.membercost END * slots AS revenue FROM Bookings LEFT JOIN Facilities ON Bookings.facid = Facilities.facid) GROUP BY name HAVING total_revenue < 1000 ORDER BY total_revenue;", conn)

,name,total_revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


We find three such facilities, the table tennis, snooker, and pool tables.

**Q11.** Produce a report of members and who recommended them in alphabetic surname,firstname order.

In [12]:
pd.read_sql_query("SELECT First.surname || ', ' || First.firstname AS member_name, Second.surname || ', ' || Second.firstname AS recommender_name FROM Members AS First LEFT JOIN Members AS Second on First.recommendedby = Second.memid WHERE First.memid > 0 ORDER BY member_name", conn)

,member_name,recommender_name
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Farrell, David",None
9,"Farrell, Jemima",None


**Q12.** Find the facilities with their usage by member, but not guests.

In [13]:
pd.read_sql_query("SELECT Facilities.name AS facility_name, Members.surname || ', ' || Members.firstname AS member_name, total_slots_booked FROM (SELECT facid, memid, SUM(slots) AS total_slots_booked FROM Bookings WHERE memid > 0 GROUP BY facid, memid) AS bookings_by_guest LEFT JOIN Facilities ON bookings_by_guest.facid = Facilities.facid LEFT JOIN Members ON bookings_by_guest.memid = Members.memid;", conn)

,facility_name,member_name,total_slots_booked
0,Tennis Court 1,"Smith, Tracy",93
1,Tennis Court 1,"Rownam, Tim",18
2,Tennis Court 1,"Joplette, Janice",57
3,Tennis Court 1,"Butters, Gerald",171
4,Tennis Court 1,"Tracy, Burton",93
...,...,...,...
197,Pool Table,"Rumney, Henrietta",3
198,Pool Table,"Farrell, David",25
199,Pool Table,"Worthington-Smyth, Henry",37
200,Pool Table,"Purview, Millicent",5


**Q13.** Find the facilities usage by month, but not guests.

In [14]:
pd.read_sql_query("SELECT Facilities.name, month, total_slots_booked FROM (SELECT facid, strftime('%m', starttime) AS month, SUM(slots) AS total_slots_booked FROM Bookings WHERE memid > 0 GROUP BY facid, month) AS bookings_by_month LEFT JOIN Facilities ON bookings_by_month.facid = Facilities.facid;", conn)

,name,month,total_slots_booked
0,Tennis Court 1,07,201
1,Tennis Court 1,08,339
2,Tennis Court 1,09,417
3,Tennis Court 2,07,123
4,Tennis Court 2,08,345
5,Tennis Court 2,09,414
6,Badminton Court,07,165
7,Badminton Court,08,414
8,Badminton Court,09,507
9,Table Tennis,07,98
